In [ ]:
import torch
print(torch.__version__)

1.12.1+cu113


In [ ]:
!pip install torch-scatter 
!pip install torch-sparse 
!pip install torch-cluster 
!pip install torch-spline-conv 
!pip install torch-geometric

In [ ]:
!pip install glycowork
import glycowork

In [ ]:
#export
from glycowork.ml.model_training import *
from glycowork.ml.models import *
from glycowork.ml.processing import *
from glycowork.ml.inference import *
from glycowork.ml.train_test_split import *
from glycowork.ml import models
from glycowork.ml import model_training
#export
from glycowork.glycan_data.loader import *
from glycowork.glycan_data.data_entry import *
from glycowork.motif.analysis import plot_embeddings, make_heatmap, characterize_monosaccharide, get_pvals_motifs

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install nbdev
#hide
#default_exp ml
import warnings
warnings.filterwarnings("ignore")
from nbdev.showdoc import show_doc
from IPython.display import HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#!pip install fair-esm
#import esm
#model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

**Code**

Ideally, we would be reading the proteins, sequences off of a file as to create a large dataframe, which would lend to easier naming, but for now, using a number will suffice 

In addition, any large scale scheme would need a conversion into embedding, which to save time on each run should be saved. 

In [ ]:
agr = 'PSVPAFKGHSFLAFPTLRAYHTLRLALEFRALETEGLLLYNGNARGKDFLALALLDGHVQFRFDTGSGPAVLTSLVPVEPGRWHRLELSRHWRQGTLSVDGEAPVVGESPSGTDGLNLDTKLYVGGLPEEQVATVLDRTSVGIGLKGCIRMLDINNQQLELSDWQRAVVQSSGVGEC'
# Lectin Oracle Paper's proteins
protein_SNA = 'MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADTYEPFLRALQEKVILGNHTAFDLPVLNPESQVSDSNRFVLVPLTNPSGDTVTLAIDVVNLYVVAFSSNGKSYFFSGSTAVQRDNLFVDTTQEELNFTGNYTSLERQVGFGRVYIPLGPKSLDQAISSLRTYTLTAGDTKPLARGLLVVIQMVSEAARFRYIELRIRTSITDASEFTPDLLMLSMENNWSSMSSEIQQAQPGGIFAGVVQLRDERNNSIEVTNFRRLFELTYIAVLLYGCAPVTSSSYSNNAIDAQIIKMPVFRGGEYEKVCSVVEVTRRISGWDGLCVDVRYGHYIDGNPVQLRPCGNECNQLWTFRTDGTIRWLGKCLTASSSVMIYDCNTVPPEATKWVVSIDGTITNPHSGLVLTAPQAAEGTALSLENNIHAARQGWTVGDVEPLVTFIVGYKQMCLRENGENNFVWLEDCVLNRVQQEWALYGDGTIRVNSNRSLCVTSEDHEPSDLIVILKCEGSGNQRWVFNTNGTISNPNAKLLMDVAQRDVSLRKIILYRPTGNPNQQWITTTHPA'
gal_7 = 'MSNVPHKSSLPEGIRPGTVLRIRGLVPPNASRFHVNLLCGEEGGSDAALHFNPRLDTSEVVFNSKEGGSWGREERGPGVPFGRGQPFEVLIIASDDGFKAVVGDAQYHHFRHRLPLARVRLVEVGGDVQLDSVRIF'
lox_1 = 'DGMQLSQVSDLLTQEQANLTHQKKKLEGQISARQQAEEASQESENELKEMIETLARKLNEKSKEQMELHHQNLNLQETLKRVANCSAPCPQDWIWHGENCYLFSSGSFNWEKSQEKCLSLDAKLLKINSTADLDFIQQAISYSSFPFWMGLSRRNPSYPWLWEDGSPLMPHLFRVRGAVSQTYPSGTCAYIQRGAVYAENCILAAFSICQKKANLRAQVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK'

In [ ]:
#list of proteins to feed, 

protein_list = [protein_SNA, gal_7, lox_1]
#possibly switch to zip (name, sequence)??
# protein_list = [(SNA, abc), (gal_7, def,), (lox_1, ghi)]
glycan = ['1,5-Anhydro-ManNAc-ol(2-4)GlcNAc1N']

In [ ]:
with open('stored_protein_embeddings.pkl', 'rb') as file:
  old_dic = pickle.load(file)

In [ ]:
model = prep_model('LectinOracle',1,trained=True)

In [ ]:
#model.load_state_dict(torch.load(trained_LectinOracle_flex))
#likely unnecessary, seems to run this command already??

In [ ]:
outprint_multi_protein = pd.DataFrame(columns = ['name', 'sequence', 'pred'])

In [ ]:
glycan = 'GlcA(β1-3)Xyl(α1-3)GlcA(β1-3)Xyl(α1-3)GlcA(β)'

#GlcA(β1-3)Xyl(α1-3)GlcA(β1-3)Xyl(α1-3)GlcA(β)
#7-hydroxy-4-methylchromen-2-one
#GlcA(β1-3)Xyl(α1-3)GlcA(β1-3)Xyl(α1-3)
#GlcA(β1-3)Xyl(α1-3)GlcA(β1-3)Xyl(α1-OH)
#GlcA(β1-3)Xyl(α1-3)GlcA(β1-3)Xyl(α1-3)GlcA(β1-OH)

In [ ]:
#multiple protein, single glycan
a=0
protein_naming = R"{protein}" 
for protein in protein_list:
  outprint_multi_protein.at[a, 'name'] = a # replace with the actual name when reading from a file
  outprint_multi_protein.at[a, 'sequence'] = protein
  outprint_multi_protein.at[a, 'pred'] = (get_lectin_preds(protein, glycan,model,old_dic).at[0,'pred'])
  a += 1
outprint_multi_protein.head()

In [ ]:
#multiple protein, single glycan
a=0
protein_naming = R"{protein}" 
for protein in protein_list:
  outprint_multi_protein.at[a, 'name'] = a # replace with the actual name when reading from a file
  outprint_multi_protein.at[a, 'sequence'] = protein
  outprint_multi_protein.at[a, 'pred'] = (get_lectin_preds(protein, glycan,model,old_dic).at[0,'pred'])
  a += 1
outprint_multi_protein.head()

In [ ]:
# single protein, multiple glycan
preds = get_lectin_preds(protein_SNA, df_glycan.glycan.tolist(),model,old_dic)

In [ ]:
#preds.to_csv()